#TODO: figure out how I can integrate the knowledge of last sleep stage into my research

# Utilities

In [ ]:
import numpy as np
import scipy as spy
import pandas as pd
from pathlib import Path
import mne
import matplotlib.pyplot as plt
import seaborn as sns
import mne
import pingouin as pg

In [ ]:
def read_dream_interpretation(path: Path):
    interpretation = pd.read_csv(path, sep=',')
    return interpretation

In [ ]:
def read_eeg(path:Path):
    raw = mne.io.read_raw_edf("./data/PSG/"+path, preload=True, verbose=False)

    data, times = raw.get_data(return_times=True)
    ch_names = raw.ch_names
    
    df = pd.DataFrame(data.T, columns=ch_names)
    df["time_s"] = times
    return df


In [ ]:
def plot_dataframe_signals(df: pd.DataFrame, title: str, max_samples = None, time_col: str = None):
    sns.set_style("whitegrid")

    df_plot = df.copy()
    
    if max_samples != None and max_samples < len(df_plot):
        new_indices = np.linspace(0, len(df_plot), max_samples, endpoint=False, dtype=np.int32)
        df_plot = df_plot.iloc[new_indices]

    if time_col and time_col in df_plot.columns:
        x = df_plot[time_col].values
        xlabel = "Time [s]"
        df_plot = df_plot.drop(columns=[time_col])
    else:
        x = np.arange(len(df_plot))
        xlabel = "Samples"
    

    numeric_cols = df_plot.select_dtypes(include=[np.number]).columns

    plt.figure(figsize=(15, 6))
    for col in numeric_cols:
        sns.lineplot(x=x, y=df_plot[col], label=col)

    plt.title(title, fontsize=14, weight="bold")
    plt.xlabel(xlabel)
    plt.ylabel("Amplitude")
    plt.legend(title="Channels", loc="best")
    plt.grid(True, which="major", linestyle="--", alpha=0.6)
    plt.grid(True, which="minor", linestyle=":", alpha=0.3)
    plt.gca().set_facecolor("lightblue")

    plt.tight_layout()
    plt.show()


In [ ]:
def plot_dataframe_fft(df: pd.DataFrame, title: str, fs: float = None, time_col: str = None):
    sns.set_style("whitegrid")
    df_plot = df.copy()

    if time_col and time_col in df_plot.columns:
        time = df_plot[time_col].values
        df_plot = df_plot.drop(columns=[time_col])
        if fs is None:
            if len(time) > 1:
                fs = 1.0 / np.mean(np.diff(time))
                print(f"Inferred sampling frequecny: {fs}")
            else:
                raise ValueError("Not enough time samples to infer sampling frequency.")
    elif fs is None:
        raise ValueError("Sampling frequency 'fs' must be provided if no time column is given.")

    numeric_cols = df_plot.select_dtypes(include=[np.number]).columns
    n = len(df_plot)

    freqs = spy.fft.rfftfreq(n, d=1/fs)

    plt.figure(figsize=(15, 6))
    for col in numeric_cols:
        sig = df_plot[col].values
        fft_vals = spy.fft.rfft(sig)
        amplitude = np.abs(fft_vals) / (n / 2)
        amplitude = 10 * np.log10(amplitude + 1e-20)
        sns.lineplot(x=freqs, y=amplitude, label=col)

    plt.title(title + " — Amplitude Spectrum", fontsize=14, weight="bold")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude [dB]")
    plt.legend(title="Channels", loc="best")
    plt.grid(True, which="major", linestyle="--", alpha=0.6)
    plt.grid(True, which="minor", linestyle=":", alpha=0.3)
    plt.gca().set_facecolor("lightblue")
    plt.tight_layout()
    plt.show()


In [ ]:
def extract_brain_frequency(df : pd.DataFrame, fs, bPass, bStop, maxLoss, minAttenuation):
    ord, w = spy.signal.cheb1ord(bPass, bStop, maxLoss, minAttenuation, analog=False, fs=fs)
    
    f_type = "bandpass"
    if len(bStop) == 1:
        if(bPass[0] < bStop[0]):
            f_type = "lowpass"
        elif(bPass[0] > bStop[0]):
            f_type = "highpass"
    
    ripple = 3
    b, a = spy.signal.cheby1(ord, ripple, w, analog=False, fs=fs, btype=f_type)
    
    print(f"Filter order: {ord}")
    
    for i, column in enumerate(df.columns):
        df.iloc[:,i] = spy.signal.lfilter(b, a, df[column])
   
    return df
    

In [ ]:
def get_subject_prefixes(subjects : np.array):
    prefixes = [x.split('.')[1] for x in subjects]
    return np.array(prefixes)

In [ ]:
def apply_window(df : pd.DataFrame):
    kaiser_window = spy.signal.windows.get_window(("kaiser", 2), len(df))
    for idx, column in enumerate(df.columns):
        df.iloc[:, idx] = kaiser_window * df[column]
        
    return df

In [ ]:
def get_rid_of_bad_channels(patient_recording_id : str, patient_recording : pd.DataFrame, recordings_meta : pd.DataFrame):
    bad_channels_str = recordings_meta[recordings_meta["Filename"] == f"{patient_recording_id}.edf"]["Bad channels"].to_string().strip()

    try:
        bad_channels = [int(k)-1 for k in bad_channels_str.split()[1:]] #channels are indexed from 1 + index must be removed
    except:
        bad_channels = []
        pass
    
    print(f"Bad eeg channels: {patient_recording.columns[bad_channels]}")
    
    good_channels = []
    for i in range(7):
        if i not in bad_channels:
            good_channels.append(patient_recording.columns[i])
            
    good_channels = sorted(good_channels)
            
    print(f"Good eeg channels: {good_channels}")
            
    patient_recording.drop(columns=patient_recording.columns[bad_channels], inplace=True)
    
    #removing line noise
    fs=250
    ord, w = spy.signal.buttord([48, 51], [49.5, 50.5], 1, 10, analog=False, fs=fs)
    print(f"Line filter order: {ord}")
    b, a = spy.signal.butter(ord, w, analog=False, fs=fs, btype="bandstop")
        
    for idx, column in enumerate(patient_recording.columns):
        patient_recording.iloc[:, idx] = spy.signal.lfilter(
            b, a, patient_recording[column].to_numpy()
        )

    
    return good_channels

In [ ]:
def plot_dreem_topomap(metrics, channel_names, fs, title="Topomap", value_label="Some stupid value"):
    """
    Plot a topomap for DREEM bipolar signals using midpoint-based electrode positions.
    """

    montage_1020 = mne.channels.make_standard_montage("standard_1020")
    pos_std = montage_1020.get_positions()["ch_pos"]

    # Compute midpoint
    def midpoint(a, b):
        return (pos_std[a] + pos_std[b]) / 2

    dreem_positions = {
        "EEG Fp1-O1": midpoint("Fp1", "O1"),
        "EEG Fp1-O2": midpoint("Fp1", "O2"),
        "EEG Fp1-F7": midpoint("Fp1", "F7"),
        "EEG F8-F7":  midpoint("F8", "F7"),
        "EEG F7-O1": midpoint("F7", "O1"),
        "EEG F8-O2": midpoint("F8", "O2"),
        "EEG Fp1-F8": midpoint("Fp1", "F8")
    }

    used_positions = {ch: dreem_positions[ch] for ch in channel_names}

    dig_montage = mne.channels.make_dig_montage(
        ch_pos=used_positions,
        coord_frame="head"
    )

    info = mne.create_info(
        ch_names=list(used_positions.keys()),
        sfreq=fs,
        ch_types="eeg"
    )
    info.set_montage(dig_montage)

    fig, ax = plt.subplots(figsize=(6, 5))
    im, _ = mne.viz.plot_topomap(
        np.asarray(metrics),
        pos=info,             
        axes=ax,
        cmap="viridis",
        show=False
    )

    ax.set_title(title)
    plt.colorbar(im, ax=ax, label=value_label)
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mne
import re

%matplotlib widget

def animate_dreem_topomap(metrics_time,
                          channel_names,
                          fs=128,
                          title="Animated Topomap",
                          value_label="Some stupid value",
                          interval=100,
                          save=False,
                          fps=15,
                          format="gif"):
    """
    Animate topomap for DREEM bipolar EEG signals.

    Parameters
    ----------
    metrics_time : np.ndarray
        (time, n_channels)
    channel_names : list
        Channel labels
    save : bool
        Save to file if True
    format : str
        'gif' or 'mp4'
    """

    metrics_time = np.asarray(metrics_time)
    n_time, n_ch = metrics_time.shape

    if len(channel_names) != n_ch:
        raise ValueError("Number of channel names must match metrics_time.shape[1]")

    safe_title = re.sub(r'[^a-zA-Z0-9_\- ]', '', title).replace(" ", "_")

    # --------- Standard 10–20 positions ---------
    montage_1020 = mne.channels.make_standard_montage("standard_1020")
    pos_std = montage_1020.get_positions()["ch_pos"]

    def midpoint(a, b):
        return (pos_std[a] + pos_std[b]) / 2

    dreem_positions = {
        "EEG Fp1-O1": midpoint("Fp1", "O1"),
        "EEG Fp1-O2": midpoint("Fp1", "O2"),
        "EEG Fp1-F8": midpoint("Fp1", "F8"), #frontal right
        "EEG Fp1-F7": midpoint("Fp1", "F7"), #frontal left
        
        "EEG F8-F7":  midpoint("F8", "F7"),
        "EEG F8-O2":  midpoint("F8", "O2"),
        
        "EEG F7-O1":  midpoint("F7", "O1"),
    }

    used_positions = {ch: dreem_positions[ch] for ch in channel_names}

    dig_montage = mne.channels.make_dig_montage(
        ch_pos=used_positions,
        coord_frame="head"
    )

    info = mne.create_info(
        ch_names=list(used_positions.keys()),
        sfreq=fs,
        ch_types="eeg"
    )
    info.set_montage(dig_montage)

    vmin = np.min(metrics_time)
    vmax = np.max(metrics_time)

    fig, ax = plt.subplots(figsize=(6, 5))

    im, _ = mne.viz.plot_topomap(
        metrics_time[0],
        pos=info,
        axes=ax,
        show=False,
        cmap="viridis",
        vlim=(vmin, vmax)
    )

    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label(value_label)

    ax.set_title(f"{title}\nFrame: 0")

    def update(frame):
        ax.clear()

        im, _ = mne.viz.plot_topomap(
            metrics_time[frame],
            pos=info,
            axes=ax,
            show=False,
            cmap="viridis",
            vlim=(vmin, vmax)
        )

        ax.set_title(f"{title}\nFrame: {frame}")
        return [im]

    ani = animation.FuncAnimation(
        fig,
        update,
        frames=n_time,
        interval=interval,
        blit=False,
        repeat=False
    )

    if save:
        if format.lower() == "gif":
            from matplotlib.animation import PillowWriter
            writer = PillowWriter(fps=fps)
            filename = f"{safe_title}.gif"
            ani.save(filename, writer=writer)

        elif format.lower() == "mp4":
            from matplotlib.animation import FFMpegWriter
            writer = FFMpegWriter(fps=fps, bitrate=1800)
            filename = f"{safe_title}.mp4"
            ani.save(filename, writer=writer)

        else:
            raise ValueError("format must be 'gif' or 'mp4'")

    plt.show()
    return ani


In [ ]:
def compute_spectral_power(
    df: pd.DataFrame,
    fs,
    window,
    window_duration,
    overlap_ratio,
    freq_band,
    use_db=True,
    normalize_power=False,
    use_density: bool = False
) -> pd.DataFrame:
    win_point_length = int(fs * window_duration)
    win_point_overlap = int(win_point_length * overlap_ratio)

    mean_psds = {}

    for signal in df.columns:
        f, t, Sxx = spy.signal.spectrogram(
            df[signal],
            fs=fs,
            window=window,
            nperseg=win_point_length,
            noverlap=win_point_overlap,
            scaling="density",
            mode="psd",
            return_onesided=True,
        )

        f_low, f_high = freq_band
        band_mask = (f >= f_low) & (f <= f_high)
        
        if use_density:
            band_avg_psd = np.mean(Sxx[band_mask, :], axis=0)
        else:
            band_avg_psd = np.sum(Sxx[band_mask, :], axis=0) * (f[1] - f[0])

        if use_db:
            band_avg_psd = 10 * np.log10(band_avg_psd + 1e-20)
        mean_psds[signal] = band_avg_psd

    mean_as_dict = pd.DataFrame.from_dict(mean_psds)

    if normalize_power:
        mean_psd_np = (
            mean_as_dict.to_numpy()
        )  # this sucks but we are doing python after all
        norms = np.linalg.norm(mean_psd_np, axis=1)
        mean_psd_np = mean_psd_np / norms[:, None]
        mean_as_dict = pd.DataFrame(mean_psd_np)
        mean_as_dict.columns = df.columns

    mean_as_dict.set_index(t, inplace=True)

    return mean_as_dict

In [ ]:
def plot_mean_spectral_power(
    signals_df: pd.DataFrame, 
    fs: float, 
    window: str, 
    window_duration: float, 
    overlap_ratio: float, 
    freq_band: tuple, 
    use_db: bool = True,
    use_density: bool = False
):  
    win_point_length = int(fs * window_duration)
    win_point_overlap = int(win_point_length * overlap_ratio)

    sns.set_style("whitegrid")
    fig, ax = plt.subplots(1, 1, figsize=(20, 6))

    f_low, f_high = freq_band
    
    ylabel = "Average PSD [dB/Hz]" if use_db else "Average PSD [V²/Hz]"
    title = f"Average PSD in {f_low:.1f}-{f_high:.1f} Hz Band"

    for signal_name in signals_df.columns:
        signal = signals_df[signal_name].values
        
        f, t, Sxx = spy.signal.spectrogram(
            signal,
            fs=fs,
            window=window,
            nperseg=win_point_length,
            noverlap=win_point_overlap,
            scaling='density',
            mode='psd',
            return_onesided=True
        )

        band_mask = (f >= f_low) & (f <= f_high)
        if use_density:
            band_avg_psd = np.mean(Sxx[band_mask, :], axis=0)
        else:
            band_avg_psd = np.sum(Sxx[band_mask, :], axis=0) * (f[1]-f[0])
        
        if use_db:
            band_avg_psd_to_plot = 10 * np.log10(band_avg_psd + 1e-20)
        else:
            band_avg_psd_to_plot = band_avg_psd
            
        ax.plot(
            t, 
            band_avg_psd_to_plot, 
            linewidth=1.8, 
            label=signal_name
        )
        
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Time [s]", fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.set_facecolor("whitesmoke")
    ax.legend(title="Signals", loc="best") 

    plt.tight_layout()
    plt.show()

# Common

In [ ]:
interpretation = read_dream_interpretation("./data/reports_interpreted.csv")
meta = pd.read_csv("./data/Records.csv")
eeg_channels_names = sorted(
    [
        "EEG Fp1-O1",
        "EEG Fp1-O2",
        "EEG Fp1-F8",
        "EEG Fp1-F7",
        "EEG F8-F7",
        "EEG F8-O2",
        "EEG F7-O1",
    ]
)

frequencies_ranges = {
    "Delta": [[4], [5]],
    "Theta": [[4, 7], [3, 8]],
    "Alpha": [[8, 12], [7, 13]],
    "Beta": [[15, 30], [14, 31]],
    "Gamma": [[31], [30]],
}

group_to_int = {"nightmare":0, "neutral":1, "positive":2}

stage_to_int = {"N2":2, "REM":5, "N1":1}

studied_stage = stage_to_int["N2"]

In [ ]:
nightmare_subjects = interpretation[interpretation["Classification"] == "Nightmare"]
nightmare_subjects = nightmare_subjects[nightmare_subjects["Stage"] == studied_stage]
num_nightmare_subjects = len(nightmare_subjects)
nightmare_records = get_subject_prefixes(nightmare_subjects["Source File"].to_numpy())
print(f"Num nightmare entries: {num_nightmare_subjects}. \n")

neutral_subjects = interpretation[interpretation["Classification"] == "Neutral"]
neutral_subjects = neutral_subjects[neutral_subjects["Stage"] == studied_stage]
num_neutral_subjects = len(neutral_subjects)
neutral_records = get_subject_prefixes(neutral_subjects["Source File"].to_numpy())
print(f"Num neutral entries: {num_neutral_subjects}. \n")

positive_subjects = interpretation[interpretation["Classification"] == "Positive"]
positive_subjects = positive_subjects[positive_subjects["Stage"] == studied_stage]
num_positive_subjects = len(positive_subjects)
positive_records = get_subject_prefixes(positive_subjects["Source File"].to_numpy())
print(f"Num positive entries: {num_positive_subjects}. \n")

unpleasant_subjects = interpretation[interpretation["Classification"] == "Unpleasant/Anxious"]
unpleasant_subjects = unpleasant_subjects[unpleasant_subjects["Stage"] == studied_stage]
num_unpleasant_subjects = len(unpleasant_subjects)
unpleasant_records = get_subject_prefixes(unpleasant_subjects["Source File"].to_numpy())
print(f"Num unpleasant entries: {num_unpleasant_subjects}. \n")

In [ ]:
# nightmare_subjects = unpleasant_subjects
# num_nightmare_subjects = len(nightmare_subjects)
# nightmare_records = unpleasant_records

# Tests

In [ ]:
subject_name = "DL014/DL014_night03"
subject = read_eeg(f"/{subject_name}.edf")

In [ ]:
available_eeg_channels = get_rid_of_bad_channels(subject_name, subject, meta)

In [ ]:
plot_dataframe_signals(
    subject[
        available_eeg_channels
    ],
    "Test EEG",
    500,
    None,
)
plot_dataframe_signals(
    subject[["Accel-x", "Accel-y", "Accel-z", "POx-Red", "POx-InfraRed"]],
    "Test Acceleration",
    500,
    None,
)

plot_dataframe_signals(
    subject[["POx-Red", "POx-InfraRed"]],
    "Test Pulse-Oximeter",
    500,
    None,
)

In [ ]:
plot_dataframe_fft(
    subject[
        available_eeg_channels
    ],
    "Test EEG",
    250,
    None,
)
plot_dataframe_fft(
    subject[["Accel-x", "Accel-y", "Accel-z", "POx-Red", "POx-InfraRed"]],
    "Test Acceleration",
    250,
    None,
)

plot_dataframe_fft(
    subject[["POx-Red", "POx-InfraRed"]],
    "Test Pulse-Oximeter",
    250,
    None,
)

In [ ]:
plot_mean_spectral_power(subject[available_eeg_channels], 250, "hamming", 0.25, 0.25, (0.5, 35))

In [ ]:
frequency_of_interest = extract_brain_frequency(subject[available_eeg_channels], 250, [8, 12], [7,13], 2, 15)
windowed_frequency = apply_window(frequency_of_interest)

mean_psd = compute_spectral_power(windowed_frequency, 250, "hamming", 0.25, 0.25, (0.5, 35), False)

print(mean_psd.to_numpy().shape)

time_slice = mean_psd.to_numpy()[0,:]  #rows - time, cols - channels
time_slice = time_slice / np.linalg.norm(time_slice)

print(available_eeg_channels)
print(time_slice.shape)

plot_dreem_topomap(time_slice, available_eeg_channels, fs=250, title="Instantaneous mean PSD", value_label="Mean PSD (normalized)")


In [ ]:
mean_psd_np = mean_psd.to_numpy()
norms = np.linalg.norm(mean_psd_np, axis=1)

print(norms.shape)

mean_psd_np = mean_psd_np / norms[:, None]

print(mean_psd_np.shape)

animate_dreem_topomap(mean_psd_np, available_eeg_channels, fs=250, save=False, fps=5, interval=250)

# Statistics computation

In [ ]:
subject_name = "DL014/DL014_night03"
subject = read_eeg(f"/{subject_name}.edf")
available_eeg_channels = get_rid_of_bad_channels(subject_name, subject, meta)
print(available_eeg_channels)

### Mean PSD

In [ ]:
def obtain_mean_psd(bandpass, bandstop, subject_name, use_db = False, normalize = False, use_density = True):
    subject_psd = read_eeg(f"/{subject_name}.edf")
    frequency_of_interest = extract_brain_frequency(subject_psd[available_eeg_channels], 250, bandpass, bandstop, 2, 15)
    windowed_frequency = apply_window(frequency_of_interest)

    band = bandpass
    if len(bandstop) == 1:
        print("neither bandpass nor bandstop")
        if(bandpass[0] < bandstop[0]):
            band = (0, bandpass[0])
        elif(bandpass[0] > bandstop[0]):
            band = (bandpass[0], bandpass[0] + 100)
    
    mean_psd = compute_spectral_power(windowed_frequency, 250, "hamming", 0.2, 0.15, band, use_db, normalize, use_density)  
    print(mean_psd.shape)
  
    return mean_psd #rows - time, cols - channels

mean_psd = obtain_mean_psd([8, 12], [7,13], subject_name, False, True, False)
time_slice = mean_psd.to_numpy()[10,:]  #rows - time, cols - channels

plot_dreem_topomap(time_slice, available_eeg_channels, fs=250, title="Instantaneous mean PSD", value_label="Mean PSD (normalized)")

mean_psd = obtain_mean_psd([8, 12], [7,13], subject_name, True, False, False)
time_slice = mean_psd.to_numpy()[10,:]  #rows - time, cols - channels

plot_dreem_topomap(time_slice, available_eeg_channels, fs=250, title="Instantaneous mean PSD", value_label="Mean PSD (db)")

### FAA

In [ ]:
def obtain_faa(subject_name, detrend_faa = False):
    subject_faa = read_eeg(f"/{subject_name}.edf")
    available_eeg_channels = get_rid_of_bad_channels(subject_name, subject_faa, meta)
    alpha_frequency = extract_brain_frequency(subject_faa[available_eeg_channels], 250, [8, 12], [7,13], 1, 15)
    windowed_alpha = apply_window(alpha_frequency)
    mean_alpha_power = compute_spectral_power(windowed_alpha, 250, "hamming", 0.25, 0.25, (8, 12), True, normalize_power=False)[ ["EEG Fp1-F8",
    "EEG Fp1-F7"] ] #log but do not normalize

    # "EEG Fp1-F8" - #frontal right
    # "EEG Fp1-F7" - #frontal left

    faa = mean_alpha_power["EEG Fp1-F8"] - mean_alpha_power["EEG Fp1-F7"] 
    
    if detrend_faa:
        faa = spy.signal.detrend(faa, type='linear')
    
    return faa

faa = obtain_faa(subject_name)
plot_dataframe_signals( pd.DataFrame.from_dict({"FAA":faa}) ,"FAA test")


### Activation index

In [ ]:
def obtain_activation_index(subject_name):
    #compute beta power
    subject_b = read_eeg(f"/{subject_name}.edf")
    available_b_eeg_channels = get_rid_of_bad_channels(subject_name, subject_b, meta)
    subject_b = subject_b.astype("float128")
    beta_frequency = extract_brain_frequency(subject_b[available_b_eeg_channels], 250, [15, 30], [13, 32], 0.5, 16)
    windowed_beta = apply_window(beta_frequency)
    mean_beta_power = compute_spectral_power(windowed_beta, 250, "hamming", 0.25, 0.25, (15, 30), True)

    #compute delta power
    subject_d = read_eeg(f"/{subject_name}.edf")
    subject_d = subject_d.astype("float128")
    delta_frequency = extract_brain_frequency(subject_d[available_b_eeg_channels], 250, [0.5, 4], [0.007, 5], 0.5, 16)
    windowed_delta = apply_window(delta_frequency)
    mean_delta_power = compute_spectral_power(windowed_delta, 250, "hamming", 0.25, 0.25, (0.5, 4.5), True)

    ai = mean_delta_power / mean_beta_power 
    return ai
test_ai = obtain_activation_index(subject_name)
print(test_ai)

In [ ]:
def obtain_region_coherence(subject_name, bandpass, bandstop):
    subject_c = read_eeg(f"/{subject_name}.edf")
    available_c_eeg_channels = get_rid_of_bad_channels(subject_name, subject_c, meta)
    subject_c = subject_c.astype("float128")
    frequency_data = extract_brain_frequency(subject_c[available_c_eeg_channels], 250, bandpass, bandstop, 0.5, 16)
    
    coherence_dict = {}
    for o in range(len(available_c_eeg_channels)):
        for i in range(o+1, len(available_c_eeg_channels)):
            o_signal = frequency_data[available_c_eeg_channels[o]]
            i_signal = frequency_data[available_c_eeg_channels[i]]
            
            freq, coherence = spy.signal.coherence(o_signal, i_signal, fs=250, nperseg=130)
            mean_band_coherence = np.mean(coherence)
            coherence_dict[f"{available_c_eeg_channels[o]}x{available_c_eeg_channels[i]}"] = mean_band_coherence 
            
    return coherence_dict
test_coherence = obtain_region_coherence(subject_name, [10,15], [8,17])
print(test_coherence)

# Statistical tests

### FAA

In [ ]:
def compare_faa_groups(faa_nightmare, faa_neutral, alpha=0.05):

    sw_n = pg.normality(faa_nightmare, method="shapiro")
    sw_c = pg.normality(faa_neutral, method="shapiro")

    normal_n = sw_n["pval"].values[0] > alpha
    normal_c = sw_c["pval"].values[0] > alpha

    print("Shapiro-Wilk results:")
    print("Nightmare:\n", sw_n)
    print("Neutral:\n", sw_c)
    print()

    if normal_n and normal_c:
        print("Using unpaired t-test")
        res = pg.ttest(
            faa_nightmare,
            faa_neutral,
            paired=False,
            alternative="greater"
        )
    else:
        print("Using Mann-Whitney U test")
        res = pg.mwu(
            faa_nightmare,
            faa_neutral,
            alternative="greater"
        )

    return res


In [ ]:
random_neutral = np.random.randint(0, num_neutral_subjects, num_nightmare_subjects)

neutral_faa = [np.mean(obtain_faa(s_name)) for s_name in neutral_records]
nightmare_faa = [np.mean(obtain_faa(s_name)) for s_name in nightmare_records]

print(neutral_faa)
print(nightmare_faa)


In [ ]:
compare_faa_groups(nightmare_faa, neutral_faa)

### Activation index

In [ ]:
# over groups
# over brain regions

subjects = []
subject_group = []
electrode = []
activation_index = []

for i, n_subject in enumerate(nightmare_records):
    nai = obtain_activation_index(n_subject)

    available_electrodes = nai.columns
    mean_ai = np.mean(nai, 0).astype(np.float64)
    
    for k in range(len(available_electrodes)):
        subjects.append(i)
        subject_group.append(group_to_int["nightmare"])
        electrode.append(eeg_channels_names.index(available_electrodes[k]))
        activation_index.append(mean_ai[k])
        
    # for b in eeg_channels_names:
    #     if b not in available_electrodes:
    #         subjects.append(n_subject)
    #         subject_group.append("nightmare")
    #         activation_index.append(1.0)
    #         electrode.append(b)
            

for i, ne_subject in enumerate(neutral_records):
    nai = obtain_activation_index(ne_subject)

    available_electrodes = nai.columns
    mean_ai = np.mean(nai, 0).astype(np.float64)

    for k in range(len(available_electrodes)):
        subjects.append(i + num_nightmare_subjects)
        subject_group.append(group_to_int["neutral"])
        electrode.append(eeg_channels_names.index(available_electrodes[k]))
        activation_index.append(mean_ai[k])
        
    # for b in eeg_channels_names:
    #     if b not in available_electrodes:
    #         subjects.append(ne_subject)
    #         subject_group.append("neutral")
    #         activation_index.append(1.0)
    #         electrode.append(b)

anova_df = pd.DataFrame().from_dict(
    {
        "subject_id": subjects,
        "group": subject_group,
        "electrode": electrode,
        "ai":activation_index
    }
)
anova_df

In [ ]:
mixed_anova = pg.mixed_anova(
    data=anova_df,
    dv="ai",
    within="electrode",
    between="group",
    subject="subject_id"
)

mixed_anova

In [ ]:
results = []

for elec in anova_df["electrode"].unique():
    df_e = anova_df[anova_df["electrode"] == elec]

    aov = pg.anova(
        data=df_e,
        dv="ai",
        between="group",
        # detailed=True
    )
    
    row = aov[aov["Source"] == "group"].iloc[0]

    results.append({
        "electrode": eeg_channels_names[elec],
        "F": row["F"],
        "p_unc": row["p-unc"],
        "np2": row["np2"],
        "df1": row["ddof1"],
        "df2": row["ddof2"]
    })

per_electrode_anova = pd.DataFrame(results)

p_value_correction = pg.multicomp(
    per_electrode_anova["p_unc"].values,
    method="fdr_bh",
    alpha=0.05
)

per_electrode_anova["reject"] = p_value_correction[0]
per_electrode_anova["p_corr"] = p_value_correction[1]

per_electrode_anova


In [ ]:
plot_dreem_topomap(per_electrode_anova["F"].to_numpy(),per_electrode_anova["electrode"].to_numpy(), 250, "Per-electrode AI F-values", "F-value")

In [ ]:
plot_dreem_topomap(per_electrode_anova["p_corr"].to_numpy(),per_electrode_anova["electrode"].to_numpy(), 250, "Per-electrode AI p-values\n(corrected)", "p-value")

### Coherence

In [ ]:
eeg_channels_cross_prod = []

for i in range(len(eeg_channels_names)):
    for ii in range(i + 1, len(eeg_channels_names)):
        eeg_channels_cross_prod.append(
            f"{eeg_channels_names[i]}x{eeg_channels_names[ii]}"
        )
print(eeg_channels_cross_prod)

subject_names = []
frequency_types = []
coherence_e_pairs = []
coherence_values = []
group_names = []

for bidx, (brain_wave_name, (bPass, bStop)) in enumerate(frequencies_ranges.items()):
    for i, ni_subject in enumerate(nightmare_records):
        ni_coherence = obtain_region_coherence(ni_subject, bPass, bStop)
        for pair, coherence_v in ni_coherence.items():
            coherence_values.append(coherence_v.astype(np.float64))
            subject_names.append(i)
            frequency_types.append(bidx)
            coherence_e_pairs.append(eeg_channels_cross_prod.index(pair))
            group_names.append(group_to_int["nightmare"])

    for k, ne_subject in enumerate(neutral_records):
        ne_coherence = obtain_region_coherence(ne_subject, bPass, bStop)
        for pair, coherence_v in ne_coherence.items():
            coherence_values.append(coherence_v.astype(np.float64))
            subject_names.append(k + num_nightmare_subjects)
            frequency_types.append(bidx)
            coherence_e_pairs.append(eeg_channels_cross_prod.index(pair))
            group_names.append(group_to_int["neutral"])

coherence_df = pd.DataFrame.from_dict(
    {
        "group": group_names,
        "subject_id": subject_names,
        "wave_type": frequency_types,
        "electrode_pairs": coherence_e_pairs,
        "coherence": coherence_values,
    }
)
coherence_df

In [ ]:
wave_aovs = {}

for bidx, brain_wave_name in enumerate(frequencies_ranges.keys()):
    
    wave_type_df_slice = coherence_df[coherence_df["wave_type"] == bidx]
    
    wave_mixed_aov = pg.mixed_anova(
        data=wave_type_df_slice,
        dv="coherence",
        within="electrode_pairs",
        between="group",
        subject="subject_id",
    )
    print(f"Wave type: {brain_wave_name}. ANOVA: \n{wave_mixed_aov}\n\n")
    
    ###########
    
    results = []

    for elec in wave_type_df_slice["electrode_pairs"].unique():
        df_e = wave_type_df_slice[wave_type_df_slice["electrode_pairs"] == elec]

        aov = pg.anova(
            data=df_e,
            dv="coherence",
            between="group",
            # detailed=True
        )
        
        row = aov[aov["Source"] == "group"].iloc[0]

        results.append({
            "electrode_pair": eeg_channels_cross_prod[elec],
            "F": row["F"],
            "p_unc": row["p-unc"],
            "np2": row["np2"],
            "df1": row["ddof1"],
            "df2": row["ddof2"]
        })

    per_electrode_pair_anova = pd.DataFrame(results)

    p_value_correction = pg.multicomp(
        per_electrode_pair_anova["p_unc"].values,
        method="fdr_bh",
        alpha=0.05
    )

    per_electrode_pair_anova["reject"] = p_value_correction[0]
    per_electrode_pair_anova["p_corr"] = p_value_correction[1]
    
    wave_aovs[brain_wave_name] = (wave_mixed_aov, per_electrode_pair_anova)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_pairwise_metric_grid(
    electrodes,
    metrics,
    title,
    metric_name,
    cmap="viridis",
    vmin=None,
    vmax=None,
    annotate=False
):
    """
    Plot a symmetric pairwise metric matrix (e.g., coherence ANOVA F-values).

    Parameters
    ----------
    electrodes : list of str
        Electrode names (length N)

    metrics : np.ndarray (N,N) or dict {(ch1, ch2): value}
        Pairwise metric values

    title : str
        Plot title

    metric_name : str
        Name of metric (for colorbar label)

    cmap : str
        Matplotlib colormap

    vmin, vmax : float or None
        Color scale limits

    annotate : bool
        If True, print values in cells (not recommended for large N)
    """

    N = len(electrodes)
    if isinstance(metrics, dict):
        mat = np.full((N, N), np.nan)
        idx = {ch: i for i, ch in enumerate(electrodes)}

        for (ch1, ch2), val in metrics.items():
            i, j = idx[ch1], idx[ch2]
            mat[i, j] = val
            mat[j, i] = val 
    else:
        mat = np.asarray(metrics)
        assert mat.shape == (N, N), "Metric matrix must be NxN"

    mask = np.eye(N, dtype=bool)

    plt.figure(figsize=(1.1 * N, 1.0 * N))

    ax = sns.heatmap(
        mat,
        mask=mask,
        xticklabels=electrodes,
        yticklabels=electrodes,
        cmap=cmap,
        square=True,
        cbar_kws={"label": metric_name},
        vmin=vmin,
        vmax=vmax,
        annot=annotate,
        fmt=".2f",
        linewidths=0.5,
        linecolor="lightgray"
    )

    ax.set_title(title, fontsize=14, pad=12)
    ax.tick_params(axis="x", rotation=45)
    ax.tick_params(axis="y", rotation=0)

    plt.tight_layout()
    plt.show()


In [ ]:
def print_score_matrix(score_key, score_name, wave_indexed_df, dimension_column):
    for wave_name in frequencies_ranges.keys():
        anova_mixed, anova_per_pair = wave_indexed_df[wave_name]

        flattened_electrode_dict = {}
        for idx, pair_name in enumerate(anova_per_pair[dimension_column]):
            name_1, name_2 = pair_name.split('x')
            flattened_electrode_dict[(name_1, name_2)] = anova_per_pair[score_key][idx]

        plot_pairwise_metric_grid(eeg_channels_names, flattened_electrode_dict, f"{score_name} for one-way ANOVA\nWave type: {wave_name}", score_name)

In [ ]:
print_score_matrix("F", "F-value", wave_aovs, "electrode_pair")

In [ ]:
print_score_matrix("p_corr", "p-value", wave_aovs, "electrode_pair")

### PSD plotting

In [ ]:
subject_names = []
frequency_types = []
power_values = []
group_names = []
electrode = []

#TODO: maybe, compute tho logariothm of the power so as not to lose on precision

for bidx, (brain_wave_name, (bPass, bStop)) in enumerate(frequencies_ranges.items()):
    for i, ni_subject in enumerate(nightmare_records):
        ni_power = obtain_mean_psd(bPass, bStop, ni_subject, False, False, False)
        available_electrodes = ni_power.columns
        ni_power = np.mean(ni_power, axis=0)

        for idx, column_name in enumerate(available_electrodes):
            subject_names.append(i)
            frequency_types.append(bidx)
            group_names.append(group_to_int["nightmare"])
            power_values.append(ni_power[column_name])
            electrode.append(eeg_channels_names.index(available_electrodes[idx]))
            
    for k, ne_subject in enumerate(neutral_records):
        ne_power = obtain_mean_psd(bPass, bStop, ne_subject, False, False, False)
        available_electrodes = ne_power.columns
        ne_power = np.mean(ne_power, axis=0)

        for idx, column_name in enumerate(available_eeg_channels):
            subject_names.append(k + num_nightmare_subjects)
            frequency_types.append(bidx)
            group_names.append(group_to_int["neutral"])
            power_values.append(ne_power[column_name])
            electrode.append(eeg_channels_names.index(available_electrodes[idx]))

spectral_power_df = pd.DataFrame.from_dict(
    {
        "group": group_names,
        "subject_id": subject_names,
        "wave_type": frequency_types,
        "electrode": electrode,
        "power": power_values,
    }
)
spectral_power_df

In [ ]:
power_wave_aovs = {}

for bidx, brain_wave_name in enumerate(frequencies_ranges.keys()):
    
    wave_type_df_slice = spectral_power_df[spectral_power_df["wave_type"] == bidx]
    
    wave_mixed_aov = pg.mixed_anova(
        data=wave_type_df_slice,
        dv="power",
        within="electrode",
        between="group",
        subject="subject_id",
    )
    print(f"Wave type: {brain_wave_name}. ANOVA: \n{wave_mixed_aov}\n\n")
    
    ###########
    
    results = []

    for elec in range(len(eeg_channels_names)):
        df_e = wave_type_df_slice[wave_type_df_slice["electrode"] == elec]
        
        aov = pg.anova(
            data=df_e,
            dv="power",
            between="group",
            # detailed=True
        )
                
        row = aov[aov["Source"] == "group"].iloc[0]

        results.append({
            "electrode": eeg_channels_names[elec],
            "F": row["F"],
            "p_unc": row["p-unc"],
            "np2": row["np2"],
            "df1": row["ddof1"],
            "df2": row["ddof2"]
        })

    per_electrode_anova = pd.DataFrame(results)

    p_value_correction = pg.multicomp(
        per_electrode_anova["p_unc"].values,
        method="fdr_bh",
        alpha=0.05
    )

    per_electrode_anova["reject"] = p_value_correction[0]
    per_electrode_anova["p_corr"] = p_value_correction[1]
    
    power_wave_aovs[brain_wave_name] = (wave_mixed_aov, per_electrode_anova)

In [ ]:
mixed, per_electrode = power_wave_aovs["Beta"]
plot_dreem_topomap(per_electrode["F"].to_numpy(),per_electrode["electrode"].to_numpy(), 250, "Per-electrode Spectral Power F-values", "F-value")